In [ ]:
import requests

from os import makedirs, path
from queue import Queue
from threading import Thread

IMG_PATH = "../../imgs/arquigrafia"
makedirs(IMG_PATH, exist_ok=True)

ARQUI_URL = "https://www.arquigrafia.org.br/arquigrafia-images/IDNUM_view.jpg"

MIN_ID = 0
MAX_ID = 16000

In [ ]:
img_ids = list(range(MIN_ID, MAX_ID+2))
jobs = Queue()

for in_id in img_ids:
  jobs.put(in_id)

In [ ]:
def run_get_images(q):
  while not q.empty():
    img_id = q.get()

    murl = ARQUI_URL.replace("IDNUM", f"{img_id}")
    out_img_path = path.join(IMG_PATH, f"{img_id}.jpg")

    if not path.isfile(out_img_path):
      response = requests.get(murl, stream=True)
      if not response.ok:
        print(img_id, response)
      else:
        print(f"got {img_id}")
        with open(out_img_path, "wb") as fw:
          for block in response.iter_content(1024):
            if not block:
              break
            fw.write(block)

    q.task_done()

In [ ]:
NUM_THREADS = 8

for i in range(NUM_THREADS):
    worker = Thread(target=run_get_images, args=(jobs,))
    worker.start()

jobs.join()